In [42]:
import gdown

In [43]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: C:\Users\bgbg8\miconda\Scripts\ElectricCarData.csv

  0%|          | 0.00/8.20k [00:00<?, ?B/s]
100%|##########| 8.20k/8.20k [00:00<?, ?B/s]


In [44]:
import pandas as pd

In [45]:
data = pd.read_csv('ElectricCarData.csv')

In [46]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


In [47]:
data.isnull().sum()

Brand              0
Model              0
AccelSec           0
TopSpeed_KmH       0
Range_Km           0
Efficiency_WhKm    0
FastCharge_KmH     0
RapidCharge        0
PowerTrain         0
PlugType           0
BodyStyle          0
Segment            0
Seats              0
PriceEuro          0
dtype: int64

In [48]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
to_encode = ['Brand', 'Model', 'PowerTrain', 'PlugType', 'BodyStyle', 'Segment', 'RapidCharge']
for col in to_encode:
    data[col] = encoder.fit_transform(data[col])

data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,30,46,4.6,233,450,161,940,1,0,2,7,3,5,55480
1,31,33,10.0,160,270,167,250,1,2,2,1,2,5,30000
2,23,0,4.7,210,400,181,620,1,0,2,2,3,5,56440
3,2,101,6.8,180,360,206,560,1,2,2,6,3,5,68040
4,9,78,9.5,145,170,168,190,1,2,2,1,1,4,32997


In [49]:
import numpy as np
data.replace('-',np.nan, inplace=True)
print(data.isnull().sum())

from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
data['FastCharge_KmH'] = imputer.fit_transform(data['FastCharge_KmH'].to_numpy().reshape(-1,1))
data.isnull().sum()

Brand              0
Model              0
AccelSec           0
TopSpeed_KmH       0
Range_Km           0
Efficiency_WhKm    0
FastCharge_KmH     5
RapidCharge        0
PowerTrain         0
PlugType           0
BodyStyle          0
Segment            0
Seats              0
PriceEuro          0
dtype: int64


Brand              0
Model              0
AccelSec           0
TopSpeed_KmH       0
Range_Km           0
Efficiency_WhKm    0
FastCharge_KmH     0
RapidCharge        0
PowerTrain         0
PlugType           0
BodyStyle          0
Segment            0
Seats              0
PriceEuro          0
dtype: int64

In [50]:
from sklearn.model_selection import train_test_split


X = data.drop('PriceEuro', axis=1)
y = data['PriceEuro']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [51]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=50, min_samples_split=10, max_features=5)
tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=50, max_features=5, min_samples_split=10)

In [52]:
predictions_tree = tree.predict(X_test)
predictions_tree

array([ 45317.66666667,  55376.5       ,  85561.42857143,  70631.        ,
        55376.5       ,  38788.33333333,  53156.66666667,  33963.5       ,
        85561.42857143,  64596.        ,  38788.33333333,  34368.44444444,
        85561.42857143, 162181.5       ,  45317.66666667,  29124.33333333,
       162181.5       , 162181.5       ,  34368.44444444,  72118.        ,
        72118.        ])

In [53]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2_decision_regressor = r2_score(y_test, predictions_tree)
mse_decision_regressor = mean_squared_error(y_test, predictions_tree)
mae_decision_regressor = mean_absolute_error(y_test, predictions_tree)
print(f'R^2 score for the test set is: {r2_decision_regressor}')
print(f'MSE for the test set is: {mse_decision_regressor}')
print(f'MAE for the test set is: {mae_decision_regressor}')

R^2 score for the test set is: 0.6032439363076293
MSE for the test set is: 568098650.3742276
MAE for the test set is: 14483.925170068027


In [54]:
from xgboost import XGBRegressor
model = XGBRegressor(objective='reg:squarederror', max_depth=50, n_estimators=200, learning_rate=0.15)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.15, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=50, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [55]:
predictions_xgb = model.predict(X_test)
predictions_xgb

array([ 39722.348,  51451.414,  84726.555,  48341.797,  53253.945,
        36837.008,  49480.293,  35958.605,  98633.96 ,  63439.227,
        36015.227,  33805.4  , 114425.79 , 146497.38 ,  39722.348,
        29781.611, 180033.34 , 103241.05 ,  33393.887,  77914.555,
        54201.23 ], dtype=float32)

In [56]:
r2_xgb = r2_score(y_test, predictions_xgb)
mse_xgb = mean_squared_error(y_test, predictions_xgb)
mae_xgb = mean_absolute_error(y_test, predictions_xgb)
print(f'R^2 score for the test set is: {r2_xgb}')
print(f'MSE for the test set is: {mse_xgb}')
print(f'MAE for the test set is: {mae_xgb}')

R^2 score for the test set is: 0.5620579817457825
MSE for the test set is: 627071120.7208966
MAE for the test set is: 13114.825985863095


In [57]:
print(f'XGB model has {r2_xgb} R2 score while the Decision Tree model has {r2_decision_regressor} R2 score')
print(f'XGB model has {mse_xgb} MSE while the Decision Tree model has {mse_decision_regressor} MSE')
print(f'XGB model has {mae_xgb} MAE while the Decision Tree model has {mae_decision_regressor} MAE')

XGB model has 0.5620579817457825 R2 score while the Decision Tree model has 0.6032439363076293 R2 score
XGB model has 627071120.7208966 MSE while the Decision Tree model has 568098650.3742276 MSE
XGB model has 13114.825985863095 MAE while the Decision Tree model has 14483.925170068027 MAE
